In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import os
import re
import json
import torch
import torch.nn as nn
from torch.nn import functional as F

# ----------------
# hyperparameters
batch_size = 16
block_size = 128   # <-- CHANGED (was 32)
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ----------------

torch.manual_seed(1337)

# ----------------------------
# DATA
DATA_PATH = "/content/drive/MyDrive/rumi_small_clean.txt"

def clean_persian_text(t: str) -> str:
    # unify Yeh/Kaf
    t = t.replace('ي', 'ی').replace('ك', 'ک')
    # normalize line endings
    t = t.replace('\r\n', '\n').replace('\r', '\n')
    return t

with open(DATA_PATH, "r", encoding="utf-8") as f:
    text = clean_persian_text(f.read())

# ----------------------------
# Simple BPE (educational)

def _merge_seq(seq, pair, new_id):
    a, b = pair
    out = []
    i = 0
    L = len(seq)
    while i < L:
        if i < L - 1 and seq[i] == a and seq[i + 1] == b:
            out.append(new_id)
            i += 2
        else:
            out.append(seq[i])
            i += 1
    return out

class SimpleBPETokenizer:
    def __init__(self, id_to_token, merges, unk_token='�'):
        self.id_to_token = dict(id_to_token)
        self.token_to_id = {t: i for i, t in self.id_to_token.items()}
        self.merges = list(merges)  # (a_id, b_id, new_id)
        self.unk_token = unk_token
        self.unk_id = self.token_to_id.get(unk_token, None)

    @property
    def vocab_size(self):
        return len(self.id_to_token)

    @staticmethod
    def _pretokenize(t):
        return re.findall(r"\s+|\S+", t)

    @classmethod
    def train(cls, text, vocab_size=2048, min_pair_freq=2, return_encoded=True, unk_token='�'):
        parts = cls._pretokenize(text)

        chars = sorted(set(text))
        if unk_token not in chars:
            chars = [unk_token] + chars

        token_to_id = {ch: i for i, ch in enumerate(chars)}
        id_to_token = {i: ch for ch, i in token_to_id.items()}

        seqs = [[token_to_id.get(ch, token_to_id[unk_token]) for ch in p] for p in parts]

        merges = []
        next_id = len(id_to_token)

        while next_id < vocab_size:
            pair_counts = {}
            for seq in seqs:
                for a, b in zip(seq, seq[1:]):
                    pair_counts[(a, b)] = pair_counts.get((a, b), 0) + 1

            if not pair_counts:
                break

            (best_a, best_b), best_cnt = max(pair_counts.items(), key=lambda kv: (kv[1], kv[0]))
            if best_cnt < min_pair_freq:
                break

            new_id = next_id
            next_id += 1

            id_to_token[new_id] = id_to_token[best_a] + id_to_token[best_b]
            merges.append((best_a, best_b, new_id))

            pair = (best_a, best_b)
            seqs = [_merge_seq(seq, pair, new_id) for seq in seqs]

        tok = cls(id_to_token=id_to_token, merges=merges, unk_token=unk_token)

        if return_encoded:
            encoded = [tid for seq in seqs for tid in seq]
            return tok, encoded
        return tok

    def encode(self, s):
        parts = self._pretokenize(s)
        ids = []
        for p in parts:
            seq = [self.token_to_id.get(ch, self.unk_id) for ch in p]
            for a, b, new_id in self.merges:
                seq = _merge_seq(seq, (a, b), new_id)
            ids.extend(seq)
        return ids

    def decode(self, ids):
        return ''.join(self.id_to_token.get(i, self.unk_token) for i in ids)

    def save(self, path):
        obj = {
            "id_to_token": {str(i): t for i, t in self.id_to_token.items()},
            "merges": self.merges,
            "unk_token": self.unk_token,
        }
        with open(path, "w", encoding="utf-8") as f:
            json.dump(obj, f, ensure_ascii=False)

    @classmethod
    def load(cls, path):
        with open(path, "r", encoding="utf-8") as f:
            obj = json.load(f)
        id_to_token = {int(i): t for i, t in obj["id_to_token"].items()}
        merges = [tuple(m) for m in obj["merges"]]
        return cls(id_to_token=id_to_token, merges=merges, unk_token=obj.get("unk_token", "�"))

# ---- tokenizer: save/load
BPE_VOCAB_SIZE = 2048
BPE_MIN_PAIR_FREQ = 2
TOKENIZER_PATH = "/content/drive/MyDrive/rumi_bpe_tokenizer.json"
ENCODED_TENSOR_PATH = "/content/drive/MyDrive/rumi_encoded_2048.pt"

if os.path.exists(TOKENIZER_PATH):
    tokenizer = SimpleBPETokenizer.load(TOKENIZER_PATH)
    print("Loaded tokenizer:", TOKENIZER_PATH)
else:
    tokenizer, _ = SimpleBPETokenizer.train(
        text,
        vocab_size=BPE_VOCAB_SIZE,
        min_pair_freq=BPE_MIN_PAIR_FREQ,
        return_encoded=True,
    )
    tokenizer.save(TOKENIZER_PATH)
    print("Trained + saved tokenizer:", TOKENIZER_PATH)

# cache encoded tokens too (so it won't take minutes every run)
if os.path.exists(ENCODED_TENSOR_PATH):
    data = torch.load(ENCODED_TENSOR_PATH, map_location="cpu")
    print("Loaded encoded tokens:", ENCODED_TENSOR_PATH)
else:
    encoded_text = tokenizer.encode(text)
    data = torch.tensor(encoded_text, dtype=torch.long)
    torch.save(data, ENCODED_TENSOR_PATH)
    print("Encoded + saved tokens:", ENCODED_TENSOR_PATH)

vocab_size = tokenizer.vocab_size
encode = tokenizer.encode
decode = tokenizer.decode

# ----------------------------
# Train/val split
print("device:", device)
if device == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))
print("tokens:", len(data), "vocab_size:", vocab_size)

if len(data) <= block_size:
    raise ValueError(f"len(data)={len(data)} must be > block_size={block_size}")

n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    d = train_data if split == 'train' else val_data
    ix = torch.randint(len(d) - block_size, (batch_size,))
    x = torch.stack([d[i:i+block_size] for i in ix])
    y = torch.stack([d[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters, device='cpu')
        for k in range(eval_iters):
            X, Y = get_batch(split)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# ----------------------------
# Sampling: top-p (nucleus)

def top_p_filtering(logits, top_p=0.9):
    """
    logits: (B, vocab)
    keep the smallest set of tokens whose cumulative prob >= top_p
    """
    sorted_logits, sorted_idx = torch.sort(logits, descending=True, dim=-1)
    sorted_probs = F.softmax(sorted_logits, dim=-1)
    cumprobs = torch.cumsum(sorted_probs, dim=-1)

    # mask tokens after top_p mass
    mask = cumprobs > top_p
    mask[..., 0] = False  # always keep at least 1 token

    sorted_logits = sorted_logits.masked_fill(mask, float('-inf'))
    # scatter back to original order
    filtered = torch.zeros_like(logits).scatter(-1, sorted_idx, sorted_logits)
    return filtered

# ----------------------------
# Model

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, _ = x.shape
        k = self.key(x)      # (B,T,head_size)
        q = self.query(x)    # (B,T,head_size)

        # scale by head_size
        wei = q @ k.transpose(-2, -1) * (k.size(-1) ** -0.5)  # (B,T,T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        v = self.value(x)    # (B,T,head_size)
        out = wei @ v        # (B,T,head_size)
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)  # (B,T,n_embd)
        return self.dropout(self.proj(out))

class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)  # (B,T,n_embd)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))  # (T,n_embd)
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        loss = None
        if targets is not None:
            logits2 = logits.view(B*T, -1)
            targets2 = targets.view(B*T)
            loss = F.cross_entropy(logits2, targets2)

        return logits, loss

    def generate(self, idx, max_new_tokens, temperature=0.8, top_p=0.9):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]  # (B, vocab)

            logits = logits / max(temperature, 1e-6)
            logits = top_p_filtering(logits, top_p=top_p)

            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)

            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# ----------------------------
# Train

model = BigramLanguageModel().to(device)
print(sum(p.numel() for p in model.parameters()) / 1e6, "M parameters")

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    _, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# ----------------------------
# Generate

prompt = "بشنو این نی چون شکایت می‌کند\n"
context = torch.tensor([encode(prompt)], dtype=torch.long, device=device)
out = model.generate(context, max_new_tokens=600, temperature=0.8, top_p=0.9)[0].tolist()
print(decode(out))


Loaded tokenizer: /content/drive/MyDrive/rumi_bpe_tokenizer.json
Loaded encoded tokens: /content/drive/MyDrive/rumi_encoded_2048.pt
device: cuda
GPU: Tesla T4
tokens: 424041 vocab_size: 2048
0.47168 M parameters
step 0: train loss 7.8023, val loss 7.8046
step 100: train loss 4.0822, val loss 4.1260
step 200: train loss 4.0359, val loss 4.0902
step 300: train loss 3.9925, val loss 4.0408
step 400: train loss 3.9351, val loss 3.9966
step 500: train loss 3.9000, val loss 3.9916
step 600: train loss 3.8844, val loss 3.9598
step 700: train loss 3.8483, val loss 3.9530
step 800: train loss 3.8174, val loss 3.9158
step 900: train loss 3.8017, val loss 3.9122
step 1000: train loss 3.7761, val loss 3.9047
step 1100: train loss 3.7592, val loss 3.8900
step 1200: train loss 3.7424, val loss 3.8702
step 1300: train loss 3.7223, val loss 3.8655
step 1400: train loss 3.6937, val loss 3.8457
step 1500: train loss 3.6776, val loss 3.8386
step 1600: train loss 3.6630, val loss 3.8355
step 1700: train l